# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback,
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback,
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import SequenceTokenizer, SequentialDataset, TensorFeatureSource, TensorSchema, TensorFeatureInfo
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel,
)
from replay.models.nn.sequential.sasrec.optimized_model import OptimizedSasRec

import pandas as pd

## Getting a spark session

In [ ]:
spark_session = get_spark_session()

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [ ]:
!pip install rs-datasets

In [3]:
from rs_datasets import MovieLens

In [4]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [5]:
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
user_features.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [7]:
item_features.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [8]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
1000138,6040,858,4,0
1000153,6040,2384,4,1
999873,6040,593,5,2
1000007,6040,1961,4,3
1000192,6040,2019,5,4
...,...,...,...,...
825793,4958,2399,1,446
825438,4958,1407,5,447
825724,4958,3264,4,448
825731,4958,2634,3,449


### Split interactions into the train, validation and test datasets using LastNSplitter

In [9]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [10]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [11]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [12]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [13]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [14]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [15]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [16]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [ ]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [18]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4
MAX_EPOCHS = 1

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)],
)

trainer = L.Trainer(
    max_epochs=MAX_EPOCHS,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

/home/rnizmaylov/anaconda3/envs/replay/lib/python3.9/site-packages/lightning/fabric/accelerators/cuda.py:249: Can't initialize NVML
/home/rnizmaylov/anaconda3/envs/replay/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/rnizmaylov/anaconda3/envs/replay/lib/python3.9/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/rnizmaylov/work/replay_github_2/RePlay/examples/.checkpoints exists and is not empty.

  | Name   | Type             | Params
--------------------------------------------
0 | _model | SasRecModel      | 2.3 M 
1 | _loss  | CrossEntropyLoss | 0     
--------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.247     Total estimated mo

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/rnizmaylov/anaconda3/envs/replay/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 12: 'recall@10' reached 0.03543 (best 0.03543), saving model to '/home/rnizmaylov/work/replay_github_2/RePlay/examples/.checkpoints/epoch=0-step=12-v8.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=1` reached.


k              1        10        20         5
map     0.003642  0.011219  0.013458  0.009197
ndcg    0.003642  0.016811  0.025083  0.011836
recall  0.003642  0.035430  0.068377  0.019868



The path to the best model is saved inside checkpoint_callback

In [50]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [20]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback.

You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset.

You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside SasRecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback,
    ],
    logger=csv_logger,
    inference_mode=True,
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

In [22]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|   2789|3.2804818153381348|
|      0|    589|3.1946892738342285|
|      0|    352|3.1812856197357178|
|      0|   1245| 3.081799268722534|
|      0|   1178|3.0510811805725098|
|      0|   2647| 3.010064125061035|
|      0|    476| 3.007861614227295|
|      0|   2502|2.9761462211608887|
|      0|   1196| 2.946427822113037|
|      0|   1539|2.9125137329101562|
|      0|   1180|2.9087088108062744|
|      0|    315| 2.892077684402466|
|      0|   2327| 2.891298294067383|
|      0|    585|2.8746020793914795|
|      0|   1220|2.8705568313598633|
|      0|    373| 2.836061954498291|
|      0|   2559|2.8354251384735107|
|      0|   2928|2.8302085399627686|
|      0|    847|2.8144702911376953|
|      0|    108|2.8124661445617676|
+-------+-------+------------------+
only showing top 20 rows



In [23]:
pandas_res

,user_id,item_id,score
0,0,2789,3.280482
0,0,589,3.194689
0,0,352,3.181286
0,0,1245,3.081799
0,0,1178,3.051081
...,...,...,...
6039,6039,2252,2.081676
6039,6039,1720,2.081535
6039,6039,1937,2.080524
6039,6039,1262,2.077297


In [24]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([2789,  589,  352, 1245, 1178, 2647,  476, 2502, 1196, 1539, 1180,  315,
        2327,  585, 1220,  373, 2559, 2928,  847,  108,   49, 1182, 1287,  453,
        3509, 1120, 1568, 1575, 1195, 1366, 3106, 1239, 1931,  537, 1271,  900,
         912,  770, 2890, 1192, 1111,  293, 2918, 2614, 1726, 1353, 1207, 1023,
        3724, 2530,  740, 1227, 1656, 1854, 3107, 1335, 3402, 1284, 2105, 2588,
        1058, 1202, 2637, 3684, 3458, 1899, 1543, 2847, 1238, 1373, 2641,  220,
        3349, 1211,  588, 1372, 1505, 3554, 2916, 3412,  642, 2460,  376, 1885,
        1337, 1203, 3078,   46,  586,  363, 1533, 1258, 2046,   33, 2631,   20,
         892,  583,  353, 2255]) tensor([3.2805, 3.1947, 3.1813, 3.0818, 3.0511, 3.0101, 3.0079, 2.9761, 2.9464,
        2.9125, 2.9087, 2.8921, 2.8913, 2.8746, 2.8706, 2.8361, 2.8354, 2.8302,
        2.8145, 2.8125, 2.8086, 2.7917, 2.7683, 2.7613, 2.7607, 2.7484, 2.7455,
        2.7439, 2.7386, 2.7353, 2.7350, 2.7133, 2.7021, 2.6962, 2.6935, 2.691

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [25]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [26]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
|3.2804818153381348|      1|   2858|
|3.1946892738342285|      1|    593|
|3.1812856197357178|      1|    356|
| 3.081799268722534|      1|   1265|
|3.0510811805725098|      1|   1196|
| 3.010064125061035|      1|   2716|
| 3.007861614227295|      1|    480|
|2.9761462211608887|      1|   2571|
| 2.946427822113037|      1|   1214|
|2.9125137329101562|      1|   1580|
|2.9087088108062744|      1|   1198|
| 2.892077684402466|      1|    318|
| 2.891298294067383|      1|   2396|
|2.8746020793914795|      1|    589|
|2.8705568313598633|      1|   1240|
| 2.836061954498291|      1|    377|
|2.8354251384735107|      1|   2628|
|2.8302085399627686|      1|   2997|
|2.8144702911376953|      1|    858|
|2.8124661445617676|      1|    110|
+------------------+-------+-------+
only showing top 20 rows



#### Run inference on a subset of items
It happens that it is necessary to make an inference not on all items, but on a certain subset (we will call it ``candidates``). For example, you want to make predictions only for cartoons among all possible movies. 


To speed up the inference in this case, you can use the SasRec's property ``candidates_to_score``. It should be a ``torch.LongTensor`` with the IDs of the objects on which you want to make an inference. It is important that the candidate scores will be returned in the order in which their IDs were in the ``candidates_to_score``.

To use the postprocessor in this case, candidates must also be passed in it.

In [46]:
best_model_candidates = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
TOPK = [1, 10, 20, 100]
CANDIDATES = torch.range(0, 150 - 1, 1, dtype=torch.long)

postprocessors = [RemoveSeenItems(sequential_test_dataset, CANDIDATES)]

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

trainer = L.Trainer(callbacks=[pandas_prediction_callback], logger=csv_logger, inference_mode=True)
best_model_candidates.candidates_to_score = CANDIDATES
trainer.predict(best_model_candidates, dataloaders=prediction_dataloader, return_predictions=False)

There will be scores only for items whose IDs are contained in ``candidates_to_score``.

In [49]:
pandas_prediction_callback.get_result()

,user_id,item_id,score
0,0,108,2.812466
0,0,49,2.808601
0,0,46,2.353617
0,0,33,2.318751
0,0,20,2.312078
...,...,...,...
6039,6039,82,-2.342433
6039,6039,112,-2.347512
6039,6039,117,-2.364775
6039,6039,53,-2.427386


### Calculating metrics

In [29]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [30]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [31]:
metrics_to_df(result_metrics)

k,1,10,100,20
HitRate,0.00447,0.036755,0.231291,0.066722
MAP,0.00447,0.012175,0.017730,0.014208
MRR,0.00447,0.012175,0.017730,0.014208
NDCG,0.00447,0.017844,0.054526,0.025362
Precision,0.00447,0.003675,0.002313,0.003336
Recall,0.00447,0.036755,0.231291,0.066722


### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches: 

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [32]:
user_embeddings

tensor([[-1.3186, -1.8060,  1.5430,  ...,  1.6087, -0.6514,  0.1332],
        [-1.3491, -1.7284,  1.6136,  ...,  1.4948, -0.5469,  0.2030],
        [-1.3567, -1.7195,  1.5699,  ...,  1.5537, -0.5606,  0.2477],
        ...,
        [-1.4081, -1.7931,  1.5632,  ...,  1.5025, -0.5591,  0.2150],
        [-1.3812, -1.7974,  1.5736,  ...,  1.6069, -0.6063,  0.1472],
        [-1.3042, -1.7804,  1.5375,  ...,  1.5579, -0.6304,  0.0802]])

In [33]:
user_embeddings.shape

torch.Size([6040, 300])

You can access user embeddings directly with `SasRecModel` class

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = SasRecModel(tensor_schema, num_blocks=2, num_heads=2, max_len=MAX_SEQ_LEN, hidden_size=300, dropout=0.5)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data
data = next(iter(prediction_dataloader))
tensor_map, padding_mask = data.features, data.padding_mask

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask)
user_embeddings_batch

tensor([[-0.2244,  1.2574, -0.9910,  ...,  0.7930,  0.7219, -1.5780],
        [ 0.1913, -0.1102, -0.8389,  ..., -1.0506, -0.0387, -1.2965],
        [ 0.5758,  1.1741, -2.0116,  ...,  0.1773, -0.8684, -0.9513],
        ...,
        [-1.1555, -0.4980, -0.5677,  ..., -0.1869, -0.3926,  0.0411],
        [-0.5686,  0.7165,  1.0018,  ...,  0.8847, -0.0769, -0.6511],
        [ 0.5003,  1.0699, -1.4693,  ...,  0.4876, -1.1269, -0.7192]],
       grad_fn=<SliceBackward0>)

In [35]:
user_embeddings_batch.shape

torch.Size([512, 300])

### Item embeddings

`get_all_embeddings()` method in transformers can be used to get copies of all embeddings that are presented in model as a dict.

In [36]:
all_embeddings = best_model.get_all_embeddings()
all_embeddings

{'item_embedding': tensor([[ 0.0058, -0.0085,  0.0056,  ...,  0.0306,  0.0137, -0.0013],
         [-0.0142, -0.0277, -0.0037,  ...,  0.0223,  0.0028, -0.0087],
         [-0.0006,  0.0102, -0.0050,  ..., -0.0031,  0.0031, -0.0031],
         ...,
         [-0.0130,  0.0225,  0.0224,  ..., -0.0397, -0.0021, -0.0356],
         [ 0.0087,  0.0010, -0.0492,  ...,  0.0109,  0.0149, -0.0275],
         [-0.0019, -0.0160,  0.0093,  ..., -0.0444, -0.0348,  0.0209]]),
 'positional_embedding': tensor([[ 0.0722,  0.0239, -0.0339,  ...,  0.0988,  0.0660,  0.0219],
         [-0.0375,  0.0446, -0.0668,  ...,  0.0188, -0.0187, -0.0034],
         [ 0.0993,  0.0104,  0.0268,  ...,  0.1200, -0.0741,  0.0225],
         ...,
         [-0.0030, -0.0770, -0.0745,  ...,  0.0547,  0.0468, -0.0111],
         [-0.0833,  0.0804,  0.1370,  ...,  0.0096, -0.1145,  0.0691],
         [ 0.0040, -0.0250, -0.0277,  ...,  0.0556, -0.0134,  0.1312]])}

You can access item embeddings from this dictionary

In [37]:
item_embeddings = all_embeddings["item_embedding"]
item_embeddings

tensor([[ 0.0058, -0.0085,  0.0056,  ...,  0.0306,  0.0137, -0.0013],
        [-0.0142, -0.0277, -0.0037,  ...,  0.0223,  0.0028, -0.0087],
        [-0.0006,  0.0102, -0.0050,  ..., -0.0031,  0.0031, -0.0031],
        ...,
        [-0.0130,  0.0225,  0.0224,  ..., -0.0397, -0.0021, -0.0356],
        [ 0.0087,  0.0010, -0.0492,  ...,  0.0109,  0.0149, -0.0275],
        [-0.0019, -0.0160,  0.0093,  ..., -0.0444, -0.0348,  0.0209]])

Item embeddings shape is (N_ITEMS, HIDDEN_SIZE)

In [38]:
item_embeddings.shape

torch.Size([3883, 300])

Ensure we got correct dimension and ensure we got the copy of tensor

In [39]:
assert item_embeddings.shape[0] == len(tokenizer.item_id_encoder.mapping["item_id"])
assert id(item_embeddings) != id(best_model._model.item_embedder.item_emb.weight.data)

For example we observe one new item id in our training data. We can easily expand our item embedder by one element

In order to expand item embeddings by new size `set_item_embeddings_by_size` method is applied

In [40]:
best_model.set_item_embeddings_by_size(item_embeddings.shape[0] + 1)

Now our new item embeddings have one extra embedding

In [41]:
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]
old_size = item_embeddings.shape[0]

assert new_size == old_size + 1

Alternatively, we can pass our item embeddings that replace the existing ones by calling `set_item_embeddings_by_tensor`.

If tensor contains new items, they will be added to item embedder.

In [42]:
new_embeddings_weights = torch.rand((new_size + 1, 300))  # randint used for example only

best_model.set_item_embeddings_by_tensor(new_embeddings_weights)

At the moment we expanded our item embeddings by one more item and replace weights by passing `new_embeddings_weights`

In [43]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

Similarly, we can append tensor for only new items with no replace for existing by calling `append_item_embeddings`

In [44]:
new_item_weights = torch.rand((1, 300))  # randint used for example only

best_model.append_item_embeddings(new_item_weights)

We passed one new example and its weights to item embeddings, thus expanded our vocabulary by one item again

In [45]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

You can use ``candidates_to_score`` here as well.

#### Create sequence and padding_mask

In [51]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

#### Wrapping created tensors in the SasRecPredictionBatch entity

In [52]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

### Run predict step of the SasRec and get scores from the model

In [53]:
with torch.no_grad():
    scores = best_model.predict_step(batch, 0)
scores

tensor([[ 2.3214,  1.4386,  0.6909,  ..., -1.9150, -2.0709,  0.7784]])

### Getting three items with the highest score

In [54]:
torch.topk(scores, k=3).indices

tensor([[257, 352, 589]])

### Optimized inference on CPU with OpenVino
SasRec model can be compiled into IR format of OpenVino for faster inference on CPU.

SasRec model itself or the path to the checkpoint of the model can be passed as ``model`` parameter .

Parameter ``mode`` defines inference mode and shape of inputs. Could be one of ``one_query``, ``batch``, ``dynamic_batch_size``. This parameter determines whether the first dimension of the input (the batch size) will be static or dynamic.

Parameter ``num_candidates_to_score`` defines number of item ids to calculate scores if it is necessary. This parameter determines whether the model will make a partial inference and, if so, whether the list of candidates will have a static or dynamic length.

Parameter ``num_threads`` defines number of CPU threads to use.

In [55]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

Compile model from SasRec model or checkpoint.

In [56]:
opt_model = OptimizedSasRec.compile(
    model=best_model,  # or checkpoint_callback.best_model_path
    mode="one_query",
)

Wrapping tensors in the SasRecPredictionBatch entity

In [57]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

In [58]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

Run predict and get scores from the model

In [59]:
opt_model.predict(batch).shape

torch.Size([1, 3883])

#### Compiled model also supports inference on submitted candidates.

Wrapping created tensors in the SasRecPredictionBatch entity

In [60]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()},
)

Compile model and define ``num_candidates_to_score``.

In [61]:
opt_model = OptimizedSasRec.compile(
    model=best_model,
    mode="one_query",
    num_candidates_to_score=150,
)

Run predict and get scores from the model

In [62]:
opt_model.predict(batch, CANDIDATES).shape

torch.Size([1, 150])